In [1]:
#Import necessary libraries
import numpy as np #This works with numbers
import pandas as pd #This allows us to work with a dataset
import matplotlib.pyplot as plt #This allows us to plot data
from sklearn.model_selection import train_test_split #For splitting data for training and test
from sklearn.preprocessing import StandardScaler #For scaling features
from sklearn import datasets, svm 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score #For metrics and evaluation algorithm
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier #For KNN
pd.set_option('display.max_columns', None)

In [2]:
#load data into dataframe
data = pd.read_csv('HCV-Egy-Data.csv')

#dropping histoligical baseline due to duplicate representation of data
#data = data.drop('Baseline histological Grading', axis = 1)

In [3]:

#----------DESCRITIZATION

#list of columns for binning
weeks_list = ['AST 1', 'ALT 1', 'ALT4', 'ALT 12', 'ALT 24', 'ALT 36', 'ALT 48', 'ALT after 24 w']

#descritizes the columns listed in weeks_list #sources: https://dfrieds.com/data-analysis/bin-values-python-pandas.html , https://stackoverflow.com/questions/48248731/pandas-cut-multiple-columns
#for feature in weeks_list:
#    data[feature] = pd.cut(x=data[feature], bins=[0,20,40,128], labels=[1,2,3]) #discrete values    

#customized binning for the following features    
#data['Age '] = pd.cut(x=data['Age '], bins=[0,32,37,42,47,52,57,62], labels=[1,2,3,4,5,6,7])
data['BMI'] = pd.cut(x=data['BMI'], bins=[0,18.5,25,30,35,40], labels=[1,2,3,4,5])
data['WBC'] = pd.cut(x=data['WBC'], bins=[0,4000,11000,12101], labels=[1,2,3])
data['RBC'] = pd.cut(x=data['RBC'], bins=[0,3000000,5000000,5018451], labels=[1,2,3])
#data['Plat'] = pd.cut(x=data['Plat'], bins=[93013,100000,226465], labels=[1,2]) #removed a typo discrete value of 255000
data['RNA Base'] = pd.cut(x=data['RNA Base'], bins=[0,5,1201086], labels=[1,2])                                        
data['RNA 4'] = pd.cut(x=data['RNA 4'], bins=[0,5,1201715], labels=[1,2])    
data['RNA 12'] = pd.cut(x=data['RNA 12'], bins=[0,5,3731527], labels=[1,2])    
data['RNA EOT'] = pd.cut(x=data['RNA EOT'], bins=[0,5,808450], labels=[1,2])    
data['RNA EF'] = pd.cut(x=data['RNA EF'], bins=[0,5,810333], labels=[1,2])    # corrected maximum from 808450 in discretization description

# conditional binning for HGB by Gender. Each gender row is placed in a gender conditional dataframe
male_df = data[data['Gender']==1]
male_df['HGB'] = pd.cut(x=male_df['HGB'], bins=[2,14,17.5,20], labels=[1,2,3])
#print(male_df)
female_df = data[data['Gender']==2]
female_df['HGB'] = pd.cut(x=female_df['HGB'], bins=[2,12.3,15.3,20], labels=[1,2,3])
#print(female_df)
#merges male and female selections into a new dataframe
data = pd.merge(male_df, female_df, how='outer')


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
#----- dropping problematic features - Robert

#data = data.drop('Plat', axis = 1) #removed 'plat'
#dropping an index column - Robert
#data = data.drop('Unnamed: 0', axis = 1)

#----
data.describe()

,Age,Gender,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain,Plat,AST 1,ALT 1,ALT4,ALT 12,ALT 24,ALT 36,ALT 48,ALT after 24 w,Baseline histological Grading,Baselinehistological staging
count,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.00000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000
mean,46.319134,1.489531,1.515523,1.502527,1.496029,1.502527,1.498917,1.501083,1.503971,158348.06065,82.774729,83.916245,83.405776,83.510469,83.709025,83.117690,83.629603,33.438267,9.761733,2.536462
std,8.781506,0.500071,0.499939,0.500174,0.500165,0.500174,0.500179,0.500179,0.500165,38794.78555,25.993242,25.922800,26.529730,26.064478,26.205994,26.399031,26.223955,7.073569,4.023896,1.121392
min,32.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,93013.00000,39.000000,39.000000,39.000000,39.000000,39.000000,5.000000,5.000000,5.000000,3.000000,1.000000
25%,39.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,124479.00000,60.000000,62.000000,61.000000,60.000000,61.000000,61.000000,61.000000,28.000000,6.000000,2.000000
50%,46.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,157916.00000,83.000000,83.000000,82.000000,84.000000,83.000000,84.000000,83.000000,34.000000,10.000000,3.000000
75%,54.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,190314.00000,105.000000,106.000000,107.000000,106.000000,107.000000,106.000000,106.000000,40.000000,13.000000,4.000000
max,61.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,226464.00000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,45.000000,16.000000,4.000000


In [5]:
#seperate X and y
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#add a FIB-4
fib4 = (X['Age ']*X['AST 1'])/(np.sqrt(X['Plat']*X['ALT 1']))
modapri = (X['Age ']*X['AST 1'])/(X['Plat']*100)
liverDam = X['AST 1']/X['ALT 1']

X['fib4']=fib4
X['modapri']=modapri

ModX = X.iloc[:,0:3]

ModX['fib4'] = fib4
ModX['modapri'] = modapri
ModX['liverDam'] = liverDam

#ModX
X.describe()
#X.info()

,Age,Gender,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain,Plat,AST 1,ALT 1,ALT4,ALT 12,ALT 24,ALT 36,ALT 48,ALT after 24 w,Baseline histological Grading,fib4,modapri
count,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.00000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000
mean,46.319134,1.489531,1.515523,1.502527,1.496029,1.502527,1.498917,1.501083,1.503971,158348.06065,82.774729,83.916245,83.405776,83.510469,83.709025,83.117690,83.629603,33.438267,9.761733,1.119370,0.000258
std,8.781506,0.500071,0.499939,0.500174,0.500165,0.500174,0.500179,0.500179,0.500165,38794.78555,25.993242,25.922800,26.529730,26.064478,26.205994,26.399031,26.223955,7.073569,4.023896,0.484945,0.000120
min,32.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,93013.00000,39.000000,39.000000,39.000000,39.000000,39.000000,5.000000,5.000000,5.000000,3.000000,0.281124,0.000062
25%,39.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,124479.00000,60.000000,62.000000,61.000000,60.000000,61.000000,61.000000,61.000000,28.000000,6.000000,0.749051,0.000169
50%,46.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,157916.00000,83.000000,83.000000,82.000000,84.000000,83.000000,84.000000,83.000000,34.000000,10.000000,1.041989,0.000235
75%,54.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,190314.00000,105.000000,106.000000,107.000000,106.000000,107.000000,106.000000,106.000000,40.000000,13.000000,1.387189,0.000321
max,61.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,226464.00000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,45.000000,16.000000,3.267185,0.000759


In [6]:
#Split data into training and test data
#X_train, X_test, y_train, y_test = train_test_split(ModX, y, test_size = 0.20, random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

#Feature scale so that one feature doesn't have more influence than another
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
#print(X_train)

In [7]:
#Here we are going to make a simple SVM classifier
classifier = svm.NuSVC(nu = 0.4, kernel='rbf')
#Carry out cross validation 10 fold and find the mean of the score to get overall accuracy for train data
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print(scores)
print(scores.mean())

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

[0.25892857 0.21428571 0.27027027 0.30630631 0.26126126 0.1981982
 0.31531532 0.3        0.26363636 0.26605505]
0.26542570465735604


d:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [8]:
#train the classifier on train set and then print out its overall accuracy
classifier.fit(X_train, y_train)
print(classifier.score(X_train,y_train))

0.9972924187725631


d:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [9]:
#Determine the overall accuracy score of the test data set
print(classifier.score(X_test,y_test))

0.27075812274368233


In [10]:
#Making predictions
y_pred = classifier.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[23 22 15 26]
 [15 16 16 18]
 [16 15 16 19]
 [14 13 13 20]]
              precision    recall  f1-score   support

           1       0.34      0.27      0.30        86
           2       0.24      0.25      0.24        65
           3       0.27      0.24      0.25        66
           4       0.24      0.33      0.28        60

    accuracy                           0.27       277
   macro avg       0.27      0.27      0.27       277
weighted avg       0.28      0.27      0.27       277



In [11]:
#Training with k-nearest neighbor (KNN)
classifier2 = KNeighborsClassifier(n_neighbors = 10)
classifier2.fit(X_train, y_train)
#Making predictions
y_pred = classifier2.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[21 24 22 19]
 [14 17 20 14]
 [14 14 23 15]
 [17 12 15 16]]
              precision    recall  f1-score   support

           1       0.32      0.24      0.28        86
           2       0.25      0.26      0.26        65
           3       0.29      0.35      0.32        66
           4       0.25      0.27      0.26        60

    accuracy                           0.28       277
   macro avg       0.28      0.28      0.28       277
weighted avg       0.28      0.28      0.28       277

